In [15]:
import json
import pandas as pd

# Step 1: Upload the JSON file
from google.colab import files
uploaded = files.upload()

# Step 2: Load the JSON (assumes it's a full array like [ {...}, {...} ])
filename = list(uploaded.keys())[0]
with open(filename, "r", encoding="utf-8") as f:
    data = json.load(f)

# Step 3: Filter matching examples (where model_gen_answer == actual_answer)
matching = [
    ex for ex in data
    if str(ex.get("actual_answer", "")).strip().lower() == str(ex.get("model_gen_answer", "")).strip().lower()
]

# Step 4: Save to CSV
df = pd.DataFrame(matching)
df.to_csv("matching_examples.csv", index=False)

# Step 5: Download the result
files.download("matching_examples.csv")


Saving contextcite_en_QwenInstruct_COT.json to contextcite_en_QwenInstruct_COT (1).json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
for i, ex in enumerate(data[:10]):  # inspect first 10
    gold = str(ex.get("actual_answer", "")).strip()
    pred = str(ex.get("model_gen_answer", "")).strip()
    print(f"\nExample {i+1}")
    print(f"Gold   : {gold}")
    print(f"Predicted: {pred}")
    print(f"Match? {gold.lower() == pred.lower()}")



Example 1
Gold   : 18
Predicted: 18.0
Match? False

Example 2
Gold   : 3
Predicted: 3.0
Match? False

Example 3
Gold   : 70000
Predicted: 170000.0
Match? False

Example 4
Gold   : 540
Predicted: 3.0
Match? False

Example 5
Gold   : 20
Predicted: 20.0
Match? False

Example 6
Gold   : 64
Predicted: 16.0
Match? False

Example 7
Gold   : 460
Predicted: 460.0
Match? False

Example 8
Gold   : 13
Predicted: 12.0
Match? False

Example 9
Gold   : 18
Predicted: 1.0
Match? False

Example 10
Gold   : 125
Predicted: 1629.6
Match? False


In [17]:
import json
import pandas as pd

# Upload and load file
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

with open(filename, "r", encoding="utf-8") as f:
    data = json.load(f)

# Normalize by converting both answers to integers
def safe_int(text):
    try:
        return int(float(str(text).strip()))
    except:
        return None

# Keep only examples where integer parts match
matching = [
    ex for ex in data
    if safe_int(ex.get("actual_answer", "")) == safe_int(ex.get("model_gen_answer", ""))
]

# Convert to DataFrame and save
df = pd.DataFrame(matching)
df.to_csv("matching_by_integer.csv", index=False)

# Download
files.download("matching_by_integer.csv")
print(f"✅ Saved {len(df)} matching examples (integer-based match)")


Saving contextcite_en_QwenInstruct_COT.json to contextcite_en_QwenInstruct_COT (2).json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Saved 103 matching examples (integer-based match)


In [18]:
import pandas as pd
import json


from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

df = pd.read_csv(filename)

# ---------------------------
# Extract step-level scores
# ---------------------------
def extract_step_scores(row):
    try:
        steps = json.loads(row.replace("'", '"').replace('}"{', '}, {'))
        return {i: step["importance"] for i, step in enumerate(steps) if "Step" in step.get("text", "")}
    except:
        return {}

df["step_scores"] = df["lines_and_importance"].apply(extract_step_scores)

# ---------------------------
# Classify bias direction
# ---------------------------
def classify_bias_from_steps(step_scores):
    if not step_scores:
        return "unclear"

    max_idx = max(step_scores, key=step_scores.get)
    total_steps = len([k for k in step_scores if k >= 0])

    if max_idx == 0:
        return "correct_start_biased"
    elif max_idx == total_steps - 1:
        return "correct_end_biased"
    else:
        return "unclear"

df["accuracy_bias_class"] = df["step_scores"].apply(classify_bias_from_steps)

# ---------------------------
# Save result
# ---------------------------
df.to_csv("classified_correct_bias.csv", index=False)
files.download("classified_correct_bias.csv")

# Optional: Show summary
print("\n📊 Bias Direction Distribution:")
print(df["accuracy_bias_class"].value_counts())


Saving matching_by_integer.csv to matching_by_integer (1).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


📊 Bias Direction Distribution:
accuracy_bias_class
correct_start_biased    64
unclear                 39
Name: count, dtype: int64
